In [ ]:
import numpy as np
import csv

from utils_invitro_global import *

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

set_plot_settings()

EIS

In [ ]:
"""
EIS measurement was taken from a 20x20 um^2 TiN sample.
This sample was fabricated using the same recipe that was used on the implant electrodes
"""

In [ ]:
""" load data """
fn = "EIS_20x20.txt" # taken from a 20x20 um sample
fullpath = f"{ELEC_DATA_DIR}/{fn}"

freq, mag, phase = [], [], []
scale_factor = (20*20)/(14*14) # scale by BISC electrode size: 14x14
with open(fullpath, 'r') as file:
    csvreader = csv.reader(file, delimiter="\t")
    next(csvreader)
    for row in csvreader:
        freq.append(float(row[0]))
        mag.append(float(row[3])*scale_factor)
        phase.append(float(row[4]))

freq = np.asarray(freq, dtype=np.double)
mag = np.asarray(mag, dtype=np.double)
phase = np.asarray(phase, dtype=np.double)

In [ ]:
""" interpolate impedance @ 1kHz """
f1kHz = 1e3
idx = np.where(freq < f1kHz)[0][0]
f0, f1 = freq[idx], freq[idx-1] # freq is sorted descending
z0, z1 = mag[idx], mag[idx-1]
z_1kHz = mag[idx] + (z1 - z0)/(f1 - f0)*(1e3 - f0)
print(f'{z_1kHz/1e3:.2f} kOhm @ 1kHz')

# print equivalent cap
c_equiv = 1/(2*np.pi*z_1kHz*f1kHz) # (F)
print(f'Equivalent Cap: {c_equiv/1e-9:.3f} nF')

# print equivalent charge storage capacity
# compliance voltage. circuit-side compliance is ~1.4V
# but TiN CV shows its limit is near 1.2V which is also consistent with the literature
Vcomp = 1.2 # (V)
pix_area = 4*0.0014**2 # (cm^2)

csc = c_equiv*Vcomp/pix_area
print(f'Expected charge storage capacity: {csc/1e-3:.2f} (mC/cm^2)')

In [ ]:
""" plot EIS """
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3))

ax.grid(True)
ax.tick_params(axis='both', which='major')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('|Z| ($\Omega$)')

ax.loglog(freq, mag)

ax2 = ax.twinx()
ax2.semilogx(freq, phase, '--', color='purple')
ax2.set_yticks([-30, -60, -90])
ax2.tick_params(axis='both', which='major')
ax2.set_ylabel('Phase (°)')

# fig.legend(['|Z|','Phase'], loc=(0.5, 0.68), fontsize=16)
fig.legend(['|Z|','Phase'], loc=(0.5, 0.7), fontsize=14)